<a href="https://colab.research.google.com/github/MN-21/1/blob/main/p4_FunctionalAPI_california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Median house prices for California districts derived from the 1990 census.

In [ ]:
from __future__ import print_function
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras import regularizers

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import os

In [ ]:
np.random.seed(21)
tf.random.set_seed(21)

In [ ]:
housing=fetch_california_housing()
x_train, x_test, y_train, y_test=train_test_split(housing.data, housing.target,
                                                        test_size=0.2, random_state=21 )


In [ ]:
housing.keys()

In [ ]:
df=pd.DataFrame(housing.data, columns=housing.feature_names)
df.head(2)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
input_a=keras.layers.Input(shape=[5]) #6 features from 0 to 4
input_b=keras.layers.Input(shape=[6]) #5 features from 2 to 7

hid1=keras.layers.Dense(30, activation='relu')(input_b)
hid2=keras.layers.Dense(30, activation='relu')(hid1)

concat=keras.layers.concatenate([input_a, hid2])
output=keras.layers.Dense(1)(concat)

model=keras.models.Model(inputs=[input_a, input_b], outputs=[output])

In [ ]:
keras.utils.plot_model(model,)

In [ ]:
model.summary()

In [ ]:
model.compile(loss=['mse','mse'],
              loss_weights=[0.8,0.2],
              optimizer='sgd')

checkpoint=keras.callbacks.ModelCheckpoint('p4.FunctionalAPI.california_housing.h5',
                                           save_best_only=True)

early_stopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,
                                             restore_best_weights=True)

In [ ]:
x_train_a, x_train_b= x_train[:,:5], x_train[:,2:]
x_test_a, x_test_b= x_test[:,:5], x_test[:,2:]

x_new_a, x_new_b= x_test[:2,:5], x_test[:2,2:]

In [ ]:
model.fit([x_train_a,x_train_b],[y_train, y_train],
          epochs=15,
          validation_split=0.2,
          callbacks=[checkpoint, early_stopping])
          

In [ ]:
model.evaluate([x_test_a, x_test_b],y_test)

In [ ]:
model.predict([x_new_a, x_new_b])

Adding an auxiliary output for regularization:

In [ ]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)

aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [ ]:
keras.utils.plot_model(model)

In [ ]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer='sgd')

In [ ]:
model.fit([x_train_a, x_train_b], [y_train, y_train], 
          epochs=15,
          validation_split=0.2)

In [ ]:
total_loss, main_loss, aux_loss=model.evaluate([x_test_a, x_test_b])
total_loss, main_loss, aux_loss

In [ ]:
y_pred_main, y_pred_aux= model.predict([x_new_a, x_new_b])
print('y_pred_main:', y_pred_main)
print('y_pred_aux:', y_pred_aux)